In [1]:
#!/usr/bin/env python3
"""
This script loads CSIndexBR area mappings and then fetches DBLP publications along with their citation counts.
While fetching publications for a given DBLP PID, each publication is assigned a sub-area based on its venue.
Publications that do not match any sub-area are ignored.
"""

import re
import csv
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# -------------------------------
# Functions to fetch DBLP publications and citations
# -------------------------------

def get_citation_count(doi):
    """
    Get the citation count for a given DOI using the OpenCitations API.
    """
    if not doi:
        return None
    api_url = f"https://opencitations.net/index/api/v2/citation-count/doi:{doi}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        if data and 'count' in data[0]:
            return data[0]['count']
    return None

def get_dblp_publications(pid, mapping):
    """
    Fetch publications from DBLP for the given PID and assign sub-area based on CSIndexBR mapping.
    Only publications with a matching sub-area (venue in mapping) are retained.
    """
    url = f"https://dblp.org/pid/{pid}.xml"
    print(f"\nFetching DBLP publications for PID: {pid}")
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"Could not fetch data for PID={pid}, status code={resp.status_code}")
        return []
    root = ET.fromstring(resp.content)
    publications = []
    for r_element in root.findall('r'):
        if not len(r_element):
            continue
        pub_node = r_element[0]
        title = pub_node.find('title').text if pub_node.find('title') is not None else None
        # Determine venue from either journal or booktitle tag
        journal_node = pub_node.find('journal')
        booktitle_node = pub_node.find('booktitle')
        venue = (journal_node.text if journal_node is not None 
                 else (booktitle_node.text if booktitle_node is not None else None))
        doi = None
        # Extract DOI from the "ee" element
        for ee in pub_node.findall('ee'):
            if ee.text and "doi.org" in ee.text:
                doi = ee.text.replace("https://doi.org/", "")
                break
        
        if doi and venue:
            # Check if the publication's venue exists in the CSIndexBR mapping
            if venue in mapping:
                sub_area = mapping[venue][0]
                citation_count = get_citation_count(doi)
                pub = {
                    'doi': doi,
                    'title': title,
                    'venue': venue,
                    'citation_count': citation_count,
                    'sub_area': sub_area
                }
                publications.append(pub)
                print(f"Added: '{title}' | Venue: '{venue}' | Sub-area: '{sub_area}'")
            else:
                print(f"Skipping publication '{title}' – venue '{venue}' not found in CSIndexBR mapping.")
    print(f"\nTotal publications with assigned sub-area: {len(publications)}")
    return publications

def extract_doi(doi_str):
    """
    Extract DOI from a given string.
    """
    match = re.search(r'doi:(\S+)', doi_str)
    if match:
        return match.group(1)
    return doi_str

def get_references(doi):
    """
    Fetch references (publications cited by the given DOI) using the OpenCitations API.
    """
    url = f"https://opencitations.net/index/api/v2/references/doi:{doi}"
    response = requests.get(url)
    edges = []
    if response.status_code == 200:
        data = response.json()
        for record in data:
            cited_raw = record.get("cited")
            if cited_raw:
                cleaned_doi = extract_doi(cited_raw)
                edges.append({
                    "origin_doi": doi,
                    "target_doi": cleaned_doi
                })
    else:
        print(f"Error {response.status_code} when fetching references for {doi}")
    return edges

def get_citations(doi):
    """
    Fetch citations (publications that cite the given DOI) using the OpenCitations API.
    """
    url = f"https://opencitations.net/index/api/v2/citations/doi:{doi}"
    response = requests.get(url)
    edges = []
    if response.status_code == 200:
        data = response.json()
        for record in data:
            citing_raw = record.get("citing")
            if citing_raw:
                cleaned_doi = extract_doi(citing_raw)
                edges.append({
                    "origin_doi": cleaned_doi,
                    "target_doi": doi
                })
    else:
        print(f"Error {response.status_code} when fetching citations for {doi}")
    return edges

# -------------------------------
# Functions to load and merge CSIndexBR mappings
# -------------------------------

def load_csv_mapping(url, key_index=0, value_index=1):
    """
    Load a CSV from the given URL and return a mapping dictionary from key to list of values.
    """
    mapping = {}
    print(f"Loading CSV mapping from: {url}")
    response = requests.get(url)
    if response.status_code == 200:
        reader = csv.reader(response.text.splitlines())
        for row in reader:
            if len(row) <= value_index:
                continue
            key = row[key_index].strip()
            value = row[value_index].strip()
            mapping.setdefault(key, []).append(value)
    else:
        print(f"Failed to load CSV from {url}")
    return mapping

def load_all_area_mappings():
    """
    Iterates over a hardcoded list of CSIndexBR research areas and loads the corresponding
    conference and journal mappings into a global dictionary.
    """
    print("\nLoading global CSIndexBR area mappings...")
    base_url = "https://raw.githubusercontent.com/aserg-ufmg/CSIndex/refs/heads/master/data/"
    areas = ["ai", "arch", "bio", "chi", "cse", "data", "dbis", "ds",
             "formal", "graphics", "hardware", "ir", "net", "or", "pl", "robotics",
             "se", "security", "theory", "vision"]
    
    global_mapping = {}
    for area in areas:
        # Load conference mappings
        for conf_file in [f"{area}-confs.csv", f"{area}-out-confs.csv"]:
            conf_url = base_url + conf_file
            conf_mapping = load_csv_mapping(conf_url, key_index=0, value_index=1)
            for venue, sub_areas in conf_mapping.items():
                global_mapping.setdefault(venue, []).append(area)
        # Load journal mappings
        journal_file = f"{area}-out-journals.csv"
        journal_url = base_url + journal_file
        journal_mapping = load_csv_mapping(journal_url, key_index=0, value_index=0)
        for journal, sub_areas in journal_mapping.items():
            global_mapping.setdefault(journal, []).append(area)
    print("Completed loading CSIndexBR mappings.")
    return global_mapping


def build_citations_network_df(publications):
    """
    Build a DataFrame of citation edges for each publication (DOI) in the given list.
    Each edge has origin_doi, origin_sub_area, target_doi, and target_sub_area.

    We use get_references() for outgoing edges (paper -> references),
    and get_citations() for incoming edges (citing paper -> paper).

    If a cited/citing paper is not in 'publications', its sub-area is set to None.
    """
    # Create a dictionary for quick lookup of sub-areas by DOI
    doi_to_subarea = {pub["doi"]: pub["sub_area"] for pub in publications}
    
    all_edges = []
    
    for pub in publications:
        origin_doi = pub["doi"]
        origin_sub_area = pub["sub_area"]
        
        # 1) Outgoing edges (references): origin_doi -> target_doi
        references = get_references(origin_doi)
        for ref in references:
            target_doi = ref["target_doi"]
            target_sub_area = doi_to_subarea.get(target_doi, None)
            edge = {
                "origin_doi": origin_doi,
                "origin_sub-area": origin_sub_area,
                "target_doi": target_doi,
                "target_sub-area": target_sub_area
            }
            all_edges.append(edge)
        
        # 2) Incoming edges (citations): citing_doi -> origin_doi
        citations = get_citations(origin_doi)
        for cit in citations:
            citing_doi = cit["origin_doi"]
            citing_sub_area = doi_to_subarea.get(citing_doi, None)
            edge = {
                "origin_doi": citing_doi,
                "origin_sub-area": citing_sub_area,
                "target_doi": origin_doi,
                "target_sub-area": origin_sub_area
            }
            all_edges.append(edge)
    
    # Build DataFrame; remove duplicates by all columns
    citations_network_df = pd.DataFrame(all_edges).drop_duplicates()
    
    return citations_network_df

In [2]:
if __name__ == "__main__":
    # Step 0: Load researchers' data
    # URL of the CSV file
    researchers_url = "https://raw.githubusercontent.com/aserg-ufmg/CSIndex/refs/heads/master/data/all-researchers.csv"
    researchers_df = pd.read_csv(researchers_url, names=['researcher', 'institution', 'pid'])

    # Step 1: Load CSIndexBR area mappings.
    print("Step 1: Loading CSIndexBR area mappings...")
    csindex_mapping = load_all_area_mappings()

    # Step 2: Fetch publications from DBLP for a given PID,
    #         assigning sub-area as part of the fetching process.
    pid = researchers_df['pid'][0]  # taking the first PID as an example
    print(f"\nStep 2: Fetching publications for DBLP PID: {pid} (only those with a matching sub-area)")
    publications = get_dblp_publications(pid, csindex_mapping)

    # Step 3: Display the final publications DataFrame.
    if publications:
        publication_df = pd.DataFrame(publications)
        print("\nFinal Publication DataFrame (first 5 rows):")
        print(publication_df.head())
    else:
        print("No publications with a valid sub-area were retrieved.")
        exit(0)

    # Step 4: Build the citations network DataFrame (edgelist) from the fetched publications
    print("\nStep 4: Building the citations network DataFrame...")
    citations_network_df = build_citations_network_df(publications)

    # Step 5: Display the new citations network DataFrame
    if not citations_network_df.empty:
        print("\ncitations_network_df (first 10 rows):")
        print(citations_network_df.head(10))
    else:
        print("No citation edges were found for these publications.")

Step 1: Loading CSIndexBR area mappings...

Loading global CSIndexBR area mappings...
Loading CSV mapping from: https://raw.githubusercontent.com/aserg-ufmg/CSIndex/refs/heads/master/data/ai-confs.csv
Loading CSV mapping from: https://raw.githubusercontent.com/aserg-ufmg/CSIndex/refs/heads/master/data/ai-out-confs.csv
Loading CSV mapping from: https://raw.githubusercontent.com/aserg-ufmg/CSIndex/refs/heads/master/data/ai-out-journals.csv
Loading CSV mapping from: https://raw.githubusercontent.com/aserg-ufmg/CSIndex/refs/heads/master/data/arch-confs.csv
Loading CSV mapping from: https://raw.githubusercontent.com/aserg-ufmg/CSIndex/refs/heads/master/data/arch-out-confs.csv
Loading CSV mapping from: https://raw.githubusercontent.com/aserg-ufmg/CSIndex/refs/heads/master/data/arch-out-journals.csv
Loading CSV mapping from: https://raw.githubusercontent.com/aserg-ufmg/CSIndex/refs/heads/master/data/bio-confs.csv
Loading CSV mapping from: https://raw.githubusercontent.com/aserg-ufmg/CSIndex/r

In [6]:
citations_network_df

,origin_doi,origin_sub-area,target_doi,target_sub-area
0,10.1016/j.neucom.2018.10.063,ai,10.1109/tsmcb.2011.2168604,None
1,10.1016/j.neucom.2018.10.063,ai,10.1016/j.neunet.2014.10.001,None
2,10.1016/j.neucom.2018.10.063,ai,10.1109/ijcnn.2017.7966217,None
3,10.1016/j.neucom.2018.10.063,ai,10.1155/2016/3049632,None
4,10.1016/j.neucom.2018.10.063,ai,10.1364/josaa.2.001160,None
...,...,...,...,...
170,10.1016/j.sysarc.2013.08.016,None,10.1109/IPDPS.2005.72,arch
171,10.4018/978-1-60960-086-0.ch006,None,10.1109/IPDPS.2005.72,arch
172,10.1109/cctae.2010.5543241,None,10.1109/IPDPS.2005.72,arch
173,10.1145/2611562,None,10.1109/IPDPS.2005.72,arch
